In [1]:
from xml.dom import minidom
import numpy as np
import geopy.distance
import pandas as pd
import xml.etree.ElementTree as ET
import uuid
import matplotlib.pyplot as plt

COLUMNS = ["type", "from", "to", "start_tick", "duration", "slowdown"]
TRIPS_COMPLETE_PATH = "trips_completo.xml"

tree = ET.parse(TRIPS_COMPLETE_PATH)
trips = tree.getroot()

In [2]:
selected_trips = []
edges = {}

In [3]:
df = pd.read_csv('events.csv', sep=';', names=COLUMNS)
forbidden_nodes = []

In [4]:
forbidden_nodes.append(df['from'].unique())
forbidden_nodes.append(df['to'].unique())

In [5]:
forbidden_nodes = np.array(forbidden_nodes).flatten()

In [6]:
def load_nodes(path):
    nodes = []
    events_xml = minidom.parse(path)
    map_nodes_tags = events_xml.getElementsByTagName('node')
    for n in map_nodes_tags:
        nodes.append(int(n.getAttribute('id')))
    return nodes

def load_edges(path):
    map_xml = minidom.parse(path)
    map_links_tags = map_xml.getElementsByTagName('link')
    for n in map_links_tags:
        edges[int(n.getAttribute('id'))] = [
            int(n.getAttribute('from')),
            int(n.getAttribute('to'))
        ]

In [7]:
nodes = load_nodes('map_complete.xml')
load_edges('map_complete.xml')

In [8]:
def find_start_link(origin, destination):
    for k, e in edges.items():
        if (e[0] == origin):
            return k
    return -1

In [9]:
for trip in trips.iter('trip'):
    origin = int(trip.attrib['origin'])
    destination = int(trip.attrib['destination'])
    if (origin in nodes and destination in nodes):
        selected_trips.append(trip)

In [13]:
df = pd.DataFrame(columns=["name", "origin", "destination", "link_origin", "count", "start", "mode", "uuid"])

In [27]:
counter = 0
for s in selected_trips:
    origin = int(s.attrib['origin'])
    destination = int(s.attrib['destination'])
    link_origin = int(find_start_link(origin, destination))
    
    if (link_origin == -1):
        continue
        
    if ((origin in forbidden_nodes) or (destination in forbidden_nodes)):
        continue
    skip_nodes = [303561733, 2006965204, 2465920967, 2397982708, 2397319383, 5629463536, 4483150089]
    if ((origin in skip_nodes) or (destination in skip_nodes)):
        continue
    
    name = s.attrib['name']

    old_count = int(s.attrib['count'])
    start = int(s.attrib['start'])
    mode = s.attrib['mode']
    uuid_trip = 'a'
    df.loc[counter] = [name, origin, destination, link_origin, old_count, start, mode, uuid_trip]
    counter += 1

In [16]:
df = df[(df.start >= 28800) & (df.start <= 30000)]

In [17]:
df.start = df.start - 28800

In [18]:
final_df = pd.DataFrame(columns=["name", "origin", "destination", "link_origin", "count", "start", "mode", "uuid"])    

In [19]:
df.describe()

,name,origin,destination,link_origin,count,start,mode,uuid
count,46,46,46,46,46,46,46,46
unique,46,42,43,42,29,4,1,1
top,6392,303987727,1832037297,4298,23,1,car,a
freq,1,4,2,4,8,39,46,46


In [20]:
counter = 0
for row in df.iterrows():
    access = row[1]
    repeat = access['count']
    for v in range(1, repeat):    
        final_df.loc[counter] = [
            access['name'],
            access['origin'],
            access['destination'],
            access['link_origin'],
            1,
            access['start'],
            access['mode'],
            uuid.uuid4()
        ]
        counter += 1

In [21]:
final_df.to_csv('trips-with-uuid.csv', header=False, sep=';', index=False)

In [26]:

final_df.groupby(['origin', 'destination']).count()

name  link_origin  count  start  mode  uuid
origin     destination                                             
28800718   1389167404     92           92     92     92    92    92
60624568   444959901      33           33     33     33    33    33
60624571   1942703874     33           33     33     33    33    33
60624576   165459105      33           33     33     33    33    33
           2465920967     34           34     34     34    34    34
60624596   303561733      60           60     60     60    60    60
60685898   444959652      51           51     51     51    51    51
151273011  303562326      56           56     56     56    56    56
151275042  2397982708     50           50     50     50    50    50
167401305  4479246583     18           18     18     18    18    18
172017420  661212077      92           92     92     92    92    92
301665990  477363512      94           94     94     94    94    94
302805845  302805865      39           39     39     39    39    39
302805865  300058931      54           54     54     54    54    54
302806129  142811324      21           21     21     21    21    21
303987212  2050908603     22           22     22     22    22    22
303987216  28991286       22           22     22     22    22    22
303987219  302805844      23           23     23     23    23    23
303987727  60641089       22           22     22     22    22    22
           1832037297     44           44     44     44    44    44
           2578181725     22           22     22     22    22    22
303987813  302805873      22           22     22     22    22    22
303994106  303561713      22           22     22     22    22    22
304693506  3316883634     59           59     59     59    59    59
304693511  303863452      83           83     83     83    83    83
457560075  444959652      62           62     62     62    62    62
461152738  582142         49           49     49     49    49    49
759724338  4483150089    111          111    111    111   111   111
865280507  60685603      138          138    138    138   138   138
1389167235 4484837428    109          109    109    109   109   109
1389167458 1990934598     41           41     41     41    41    41
1422484997 1363061409     49           49     49     49    49    49
1836144513 3316840685     90           90     90     90    90    90
1944434915 1832037287     12           12     12     12    12    12
1955870606 457055005      99           99     99     99    99    99
2006965192 2380750803     47           47     47     47    47    47
2006965214 4167937333     60           60     60     60    60    60
2079100814 1389167308     59           59     59     59    59    59
2812273137 303860188      14           14     14     14    14    14
3841302046 1828205658     24           24     24     24    24    24
4009424168 1117834830     94           94     94     94    94    94
4085233869 1595406109     23           23     23     23    23    23
4231130771 4484837428     28           28     28     28    28    28
4236719429 598665342      33           33     33     33    33    33
4444817988 2397319383     21           21     21     21    21    21

In [183]:
final_df.destination.unique()

array([444959652, 1595406109, 60685603, 661212077, 1389167308, 1828205658,
       1832037287, 303863452, 3316883634, 302805873, 302805844,
       2050908603, 28991286, 60641089, 303561713, 2578181725, 1832037297,
       1117834830, 4484837428, 477363512, 1990934598, 300058931,
       1363061409, 302805865, 303562326, 4167937333, 2380750803,
       165459105, 1942703874, 598665342, 444959901, 1389167404,
       4479246583, 142811324, 3316840685, 303860188, 582142], dtype=object)